In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset  # Import the Subset class
from torch.optim.lr_scheduler import CosineAnnealingLR
from torchvision import datasets, transforms



# Define the MLP model
class MLP(nn.Module):
    def __init__(self, channel, num_classes):
        super(MLP, self).__init__()
        self.fc_1 = nn.Linear(28 * 28 * 1 if channel == 1 else 32 * 32 * 3, 128)
        self.fc_2 = nn.Linear(128, 128)
        self.fc_3 = nn.Linear(128, num_classes)

    def forward(self, x):
        out = x.view(x.size(0), -1)
        out = F.relu(self.fc_1(out))
        out = F.relu(self.fc_2(out))
        out = self.fc_3(out)
        return out

# Function to initialize weights randomly
def random_weights_initialization(module):
    if isinstance(module, nn.Linear):
        nn.init.normal_(module.weight)
        nn.init.constant_(module.bias, 0)

# Function to sample a mini-batch from the dataset
def sample_mini_batch(dataset, minibatch_size):
    data_loader = DataLoader(dataset, batch_size=minibatch_size, shuffle=True)
    data_iter = iter(data_loader)
    inputs, targets = next(data_iter)
    return inputs, targets

# Gradient Matching Algorithm for synthetic dataset generation
def gradient_matching_algorithm(model, dataset, lr_condensed=0.1, num_iterations=10, num_opt_steps=1):
    optimizer_condensed = optim.SGD(model.parameters(), lr=lr_condensed)

    for iteration in range(num_iterations):
        model.apply(random_weights_initialization)

        for step in range(num_opt_steps):
            inputs, targets = sample_mini_batch(dataset, minibatch_size=256)

            optimizer_condensed.zero_grad()
            outputs = model(inputs)
            loss = F.cross_entropy(outputs, targets)

            loss.backward()
            optimizer_condensed.step()

    return model

# Function to train the model on the synthetic dataset
def train_model(model, train_loader, lr_model=0.01, num_opt_steps_model=50):
    optimizer_model = optim.SGD(model.parameters(), lr=lr_model)

    for epoch in range(num_opt_steps_model):
        model.apply(random_weights_initialization)
        for data, target in train_loader:
            optimizer_model.zero_grad()

            output = model(data)
            loss = F.cross_entropy(output, target)

            loss.backward()
            optimizer_model.step()

    return model

# Function to condense the dataset
def condense_dataset(dataset, num_images_per_class):
    condensed_indices = []
    classes = dataset.targets.unique()

    for class_label in classes:
        class_indices = (dataset.targets == class_label).nonzero(as_tuple=True)[0]
        selected_indices = class_indices[:num_images_per_class]
        condensed_indices.extend(selected_indices.tolist())

    condensed_dataset = Subset(dataset, condensed_indices)
    return condensed_dataset

# Load the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
mnist_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)

# Condense the dataset to 10 images per class
condensed_dataset = condense_dataset(mnist_dataset, num_images_per_class=10)

# Initialize the model
model = MLP(channel=1, num_classes=10)

# Use the gradient matching algorithm to generate the synthetic dataset
model = gradient_matching_algorithm(model, condensed_dataset, lr_condensed=0.1, num_iterations=10, num_opt_steps=1)

# Define DataLoader for the synthetic dataset
synthetic_loader = DataLoader(condensed_dataset, batch_size=256, shuffle=True)

# Train the model on the synthetic dataset
trained_model = train_model(model, synthetic_loader, lr_model=0.01, num_opt_steps_model=50)

# Load the MNIST testing dataset
mnist_test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

# Create a DataLoader for the testing dataset
test_loader = DataLoader(mnist_test_dataset, batch_size=64, shuffle=False)

# Put the model in evaluation mode
trained_model.eval()

correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = trained_model(inputs)
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {100 * accuracy:.2f}%')


Test Accuracy: 10.54%


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset
from torch.optim.lr_scheduler import CosineAnnealingLR
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np


# Load the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,)), transforms.Resize((28, 28))])
mnist_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)

# Condense the dataset to 10 images per class
condensed_dataset = condense_dataset(mnist_dataset, num_images_per_class=10)

# Initialize the model
model = MLP(channel=1, num_classes=10)

# Use the gradient matching algorithm to generate the synthetic dataset
model = gradient_matching_algorithm(model, condensed_dataset, lr_condensed=0.1, num_iterations=10, num_opt_steps=1)

# Define DataLoader for the synthetic dataset
synthetic_loader = DataLoader(condensed_dataset, batch_size=256, shuffle=True)

# Train the model on the synthetic dataset
trained_model = train_model(model, synthetic_loader, lr_model=0.01, num_opt_steps_model=50)

# Visualize results with smaller image size
def visualize_results(model, test_loader):
    model.eval()
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            # Visualize the first few images with smaller size
            for i in range(min(len(inputs), 5)):
                image = inputs[i][0].numpy()  # Assuming channel=1
                plt.imshow(image, cmap='gray')
                plt.title(f'Predicted: {predicted[i]}, Actual: {labels[i]}')
                plt.show()

# Load the MNIST testing dataset
mnist_test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

# Create a DataLoader for the testing dataset
test_loader = DataLoader(mnist_test_dataset, batch_size=64, shuffle=False)

# Visualize results
visualize_results(trained_model, test_loader)


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


RuntimeError: ignored